# RRT Pipeline - Google Colab Setup

Dit notebook configureert de omgeving voor het draaien van de RL pipeline op Google Colab.

**De code detecteert automatisch of je op Colab of lokaal draait!**

**Stappen:**
1. GPU checken
2. Dependencies installeren
3. Code clonen van GitHub
4. Google Drive mounten (voor data)
5. Test imports & environment detection
6. Pipeline runnen

## 1. GPU Check

In [ ]:
# Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("WARNING: No GPU found! Go to Runtime > Change runtime type > GPU")

## 2. Dependencies installeren

In [ ]:
!pip install d3rlpy optuna pandas matplotlib seaborn pyyaml scikit-learn tqdm -q
print("Dependencies installed!")

## 3. Code clonen van GitHub

In [ ]:
# Clone repository
GITHUB_REPO = "https://github.com/karel2324/Thesis.git"

import os
if not os.path.exists('Thesis'):
    !git clone {GITHUB_REPO}
    print("Repository cloned!")
else:
    print("Repository already exists, pulling latest...")
    !cd Thesis && git pull

In [ ]:
# Navigeer naar code directory
%cd Thesis/RRT_start/2_Total
!ls -la

## 4. Google Drive mounten (voor data)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive mounted!")

In [ ]:
# Verify data folder exists
# Verwachte structuur in Google Drive:
#   MyDrive/Thesis/Data/
#     derived_AUMCdb_obs7_grid8_kdigo2/
#       Reward_mpen-no_disc-10/
#         MDP/
#     derived_MIMIC_obs7_grid8_kdigo2/
#       ...

DATA_PATH = "/content/drive/MyDrive/Thesis/Data"

import os
if os.path.exists(DATA_PATH):
    print(f"Data folder found: {DATA_PATH}")
    print("\nContents:")
    !ls -la {DATA_PATH}
else:
    print(f"ERROR: Data not found at {DATA_PATH}")
    print("Upload je data naar: Google Drive > MyDrive > Thesis > Data")

## 5. Test imports & Environment Detection

In [ ]:
# Test automatic environment detection
from utils import is_colab, get_base_data_dir, load_config, get_data_paths

print(f"Running on Colab: {is_colab()}")
print(f"Data directory: {get_base_data_dir()}")

# Load config and paths
config = load_config()
paths = get_data_paths(config)

print(f"\nDataset config:")
print(f"  obs_days: {config['dataset']['obs_days']}")
print(f"  grid_hours: {config['dataset']['grid_step_hours']}")
print(f"  inclusion: {config['dataset']['inclusion']}")

print(f"\nConfigured databases: {list(paths.keys())}")
for db, db_paths in paths.items():
    mdp_exists = db_paths['mdp_dir'].exists()
    print(f"  {db}: MDP dir {'EXISTS' if mdp_exists else 'NOT FOUND'}")

## 6. (Optioneel) Config aanpassen

Als je andere settings wilt gebruiken, pas config.yaml aan:

In [ ]:
# Bekijk huidige config
!head -40 config.yaml

# Om te editen: klik op Files (links) > Thesis/RRT_start/2_Total/config.yaml
# Of gebruik:
# import yaml
# config['dataset']['obs_days'] = 28
# with open('config.yaml', 'w') as f:
#     yaml.dump(config, f)

## 7. Run Pipeline

Check eerst dat `steps` in config.yaml correct staat:
- `hpo: true` voor HPO
- `evaluation: true` voor evaluatie
- etc.

In [ ]:
# Run the full pipeline
!python run_pipeline.py

## Alternatief: Run individuele stappen

In [ ]:
# HPO only
# !python _7_hpo.py

In [ ]:
# Evaluation only
# !python _9_evaluation.py

In [ ]:
# External validation only
# !python _10_external_validation.py

## 8. Resultaten bekijken

Resultaten worden automatisch opgeslagen in Google Drive.

In [ ]:
# Check resultaten folders
from utils import load_config, get_data_paths

config = load_config()
paths = get_data_paths(config)

for db, db_paths in paths.items():
    print(f"\n{db_paths['name']} output folders:")
    reward_dir = db_paths['reward_dir']
    if reward_dir.exists():
        !ls -la {reward_dir}
    else:
        print(f"  Not found: {reward_dir}")

In [ ]:
# Of download specifieke files lokaal
from google.colab import files
# files.download('path/to/results.csv')